In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import string
from numpy.random import normal,random,seed
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, cross_val_score, train_test_split
from sklearn.metrics import log_loss
from sklearn import metrics
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
import scipy.stats as st
import pickle
import lightgbm as lgb
import xlearn as xl

In [4]:
Data = pd.read_csv("../data/train.csv")

/Users/adeshghadge/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
Data = Data.loc[:, ["ProductName", "EngineVersion", "AppVersion", "AvSigVersion", "AVProductStatesIdentifier", "HasTpm", "CountryIdentifier", "CityIdentifier", "OrganizationIdentifier", "GeoNameIdentifier", "LocaleEnglishNameIdentifier", "Platform", "Processor", "OsVer", "OsBuild", "OsSuite", "OsPlatformSubRelease", "OsBuildLab", "SkuEdition", "IsProtected", "PuaMode", "SMode", "SmartScreen", "Firewall", "UacLuaenable", "Census_MDC2FormFactor", "Census_DeviceFamily", "Census_ProcessorCoreCount", "Census_ProcessorClass", "Census_PrimaryDiskTotalCapacity", "Census_PrimaryDiskTypeName", "Census_SystemVolumeTotalCapacity", "Census_HasOpticalDiskDrive", "Census_TotalPhysicalRAM", "Census_ChassisTypeName", "Census_InternalPrimaryDiagonalDisplaySizeInInches", "Census_InternalPrimaryDisplayResolutionHorizontal", "Census_InternalPrimaryDisplayResolutionVertical", "Census_PowerPlatformRoleName", "Census_OSVersion", "Census_OSArchitecture", "Census_OSBranch", "Census_OSBuildNumber", "Census_OSBuildRevision", "Census_OSEdition", "Census_OSSkuName", "Census_OSInstallTypeName", "Census_OSWUAutoUpdateOptionsName", "Census_IsPortableOperatingSystem", "Census_GenuineStateName", "Census_ActivationChannel", "Census_IsFlightsDisabled", "Census_FlightRing", "Census_IsSecureBootEnabled", "Census_IsVirtualDevice", "Census_IsTouchEnabled", "Census_IsPenCapable", "Census_IsAlwaysOnAlwaysConnectedCapable", "Wdft_IsGamer", "HasDetections"]]

In [50]:
DataTest = pd.read_csv("test.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [115]:
DataTest = DataTest.loc[:, ["ProductName", "EngineVersion", "AppVersion", "AvSigVersion", "AVProductStatesIdentifier", "HasTpm", "CountryIdentifier", "CityIdentifier", "OrganizationIdentifier", "GeoNameIdentifier", "LocaleEnglishNameIdentifier", "Platform", "Processor", "OsVer", "OsBuild", "OsSuite", "OsPlatformSubRelease", "OsBuildLab", "SkuEdition", "IsProtected", "PuaMode", "SMode", "SmartScreen", "Firewall", "UacLuaenable", "Census_MDC2FormFactor", "Census_DeviceFamily", "Census_ProcessorCoreCount", "Census_ProcessorClass", "Census_PrimaryDiskTotalCapacity", "Census_PrimaryDiskTypeName", "Census_SystemVolumeTotalCapacity", "Census_HasOpticalDiskDrive", "Census_TotalPhysicalRAM", "Census_ChassisTypeName", "Census_InternalPrimaryDiagonalDisplaySizeInInches", "Census_InternalPrimaryDisplayResolutionHorizontal", "Census_InternalPrimaryDisplayResolutionVertical", "Census_PowerPlatformRoleName", "Census_OSVersion", "Census_OSArchitecture", "Census_OSBranch", "Census_OSBuildNumber", "Census_OSBuildRevision", "Census_OSEdition", "Census_OSSkuName", "Census_OSInstallTypeName", "Census_OSWUAutoUpdateOptionsName", "Census_IsPortableOperatingSystem", "Census_GenuineStateName", "Census_ActivationChannel", "Census_IsFlightsDisabled", "Census_FlightRing", "Census_IsSecureBootEnabled", "Census_IsVirtualDevice", "Census_IsTouchEnabled", "Census_IsPenCapable", "Census_IsAlwaysOnAlwaysConnectedCapable", "Wdft_IsGamer"]]

In [51]:
DataTestFin = DataTest.copy()

In [52]:
#le = LabelEncoder()
for column_name in DataTestFin.columns:
        if DataTestFin[column_name].dtype == object:
            DataTestFin.loc[:, column_name] = le.fit_transform(DataTestFin[column_name].astype(str))
        else:
            pass

In [53]:
DataTestFin = DataTestFin.fillna(-999)

In [54]:
DataTestFin.head()

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,...,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier
0,0,4,68,70,9261,0,7.0,0,-999.0,53447.0,...,807.0,8554.0,1,-999.0,0.0,0,0,0.0,0.0,7.0
1,1,4,67,64,9005,0,7.0,0,-999.0,53447.0,...,554.0,33105.0,1,-999.0,0.0,0,0,0.0,1.0,12.0
2,2,4,65,64,8501,0,7.0,0,-999.0,49480.0,...,556.0,63396.0,1,-999.0,0.0,0,0,0.0,1.0,11.0
3,3,4,68,70,9301,0,7.0,0,-999.0,42160.0,...,628.0,26320.0,1,0.0,0.0,0,0,0.0,0.0,10.0
4,4,4,67,64,8989,0,7.0,0,-999.0,53447.0,...,556.0,63269.0,1,-999.0,0.0,0,0,0.0,1.0,3.0


In [6]:
X = Data.drop(['HasDetections'],axis=1)
y = LabelEncoder().fit_transform(Data.HasDetections)
xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.3, random_state=0)

In [4]:
xTrain.head()

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,...,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier
7587898,d9b6a534cce32ffb4f7888441e403f44,win8defender,1.1.15200.1,4.18.1807.18075,1.275.219.0,0,7.0,0,NaN,53447.0,...,142.0,2865.0,0,0.0,0.0,0,0,0.0,1.0,11.0
3904626,7009ec663dc276c08b5b1b14ea8341df,mse,1.1.15200.1,4.9.218.0,1.275.771.0,0,7.0,0,NaN,27750.0,...,556.0,13908.0,0,NaN,0.0,0,0,0.0,0.0,11.0
8705769,f9ccd21d2c98029fb0cb0342e51aca79,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1718.0,0,7.0,0,NaN,53447.0,...,556.0,63540.0,1,NaN,0.0,0,0,0.0,1.0,1.0
1343039,2690a3610a096120d55a25dad9b18c9c,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1664.0,0,7.0,0,NaN,7945.0,...,628.0,8791.0,0,NaN,0.0,0,0,0.0,1.0,10.0
7183243,ce185eeb1ba898f72fe78f06f523da27,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1555.0,0,7.0,0,NaN,53447.0,...,444.0,67773.0,0,0.0,0.0,0,0,0.0,0.0,15.0


In [4]:
del Data
del X
del y

In [5]:
le = LabelEncoder()
for column_name in xTrain.columns:
        if xTrain[column_name].dtype == object:
            xTrain.loc[:, column_name] = le.fit_transform(xTrain[column_name].astype(str))
        else:
            pass

In [6]:
xTrain = xTrain.fillna(-999)

In [8]:
xTrain.head()

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,...,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier
7587898,5311600,4,65,57,8135,0,7.0,0,-999.0,53447.0,...,142.0,2865.0,0,0.0,0.0,0,0,0.0,1.0,11.0
3904626,2733105,1,65,106,8328,0,7.0,0,-999.0,27750.0,...,556.0,13908.0,0,-999.0,0.0,0,0,0.0,0.0,11.0
8705769,6094273,4,65,57,8088,0,7.0,0,-999.0,53447.0,...,556.0,63540.0,1,-999.0,0.0,0,0,0.0,1.0,1.0
1343039,940140,4,64,57,7680,0,7.0,0,-999.0,7945.0,...,628.0,8791.0,0,-999.0,0.0,0,0,0.0,1.0,10.0
7183243,5028447,4,65,57,8038,0,7.0,0,-999.0,53447.0,...,444.0,67773.0,0,0.0,0.0,0,0,0.0,0.0,15.0


In [7]:
for column_name in xTest.columns:
        if xTest[column_name].dtype == object:
            xTest.loc[:, column_name] = le.fit_transform(xTest[column_name].astype(str))
        else:
            pass

In [8]:
xTest = xTest.fillna(-999)

In [9]:
xTest.head()

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,...,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier
4291725,1287528,4,59,57,7629,0,7.0,0,-999.0,53447.0,...,142.0,33968.0,0,-999.0,0.0,0,0,0.0,-999.0,-999.0
5081676,1524657,4,59,57,7975,0,7.0,0,-999.0,53447.0,...,142.0,33173.0,1,-999.0,0.0,1,1,0.0,0.0,1.0
8395646,2518661,4,59,76,8049,0,7.0,0,-999.0,53447.0,...,127.0,72041.0,0,0.0,0.0,0,0,0.0,0.0,3.0
5206665,1561837,4,59,51,7650,0,7.0,0,-999.0,53447.0,...,554.0,33139.0,1,0.0,0.0,0,0,0.0,0.0,1.0
7481451,2244165,4,59,57,7806,0,7.0,0,-999.0,53447.0,...,556.0,56962.0,0,-999.0,0.0,0,0,0.0,0.0,10.0


In [ ]:
typeof(yTest)

## Random Forest

In [29]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 175, num = 3)]
# Number of features to consider at every split
max_features = ['sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 40, num = 4)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [100, 137, 175], 'max_features': ['sqrt'], 'max_depth': [10, 20, 30, 40], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True]}


In [28]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [50, 60, 70, 80],
    'max_features': [2, 3],
    'min_samples_leaf': [1, 2],
    'min_samples_split': [2, 4, 6],
    'n_estimators': [100, 200, 300, 400, 500]
}

In [30]:
rfMalware = RandomForestClassifier(n_jobs=-1, verbose=2)

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rfMalware, param_grid = random_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)

In [31]:
# Fit the grid search to the data
rfGridSearch = grid_search.fit(xTrain, yTrain)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.3min
/home/mpghare/.local/lib/python3.6/site-packages/sklearn/externals/joblib/externals/loky/backend/utils.py:55: UserWarning: Failed to kill subprocesses on this platform. Pleaseinstall psutil: https://github.com/giampaolo/psutil
  warnings.warn("Failed to kill subprocesses on this platform. Please"
/home/mpghare/.local/lib/python3.6/site-packages/sklearn/externals/joblib/externals/loky/backend/utils.py:55: UserWarning: Failed to kill subprocesses on this platform. Pleaseinstall psutil: https://github.com/giampaolo/psutil
  warnings.warn("Failed to kill subprocesses on this platform. Please"
/home/mpghare/.local/lib/python3.6/site-packages/sklearn/externals/joblib/externals/loky/backend/utils.py:55: UserWarning: Failed to kill subprocesses on this platform. Pleaseinstall psutil: https://github.com/giampaolo/psutil
  warnings.warn("Failed to kill subpro

OSError: [Errno 28] No space left on device

In [39]:
import gc
gc.collect()

3088

In [ ]:
rfPred = rfMalware.predict_proba(xTest,)

In [ ]:
grid_search.best_params_

In [ ]:
best_grid = grid_search.best_estimator_

In [ ]:
rfTunedPred = best_grid.predict_proba(xTest,)

In [ ]:
log_loss(yTest,rfTunedPred)

In [ ]:
# save the model to disk
filename = 'RF_model.sav'
pickle.dump(grid_search, open(filename, 'wb'))

## Ensemble RF Models

In [12]:
estimators = []

rfclassifer1 = RandomForestClassifier(n_estimators = 100,
               bootstrap = True,random_state=6,n_jobs=-1, verbose=2)

estimators.append(('rf1', rfclassifer1))

rfclassifer2 = RandomForestClassifier(n_estimators = 120,
               bootstrap = True,random_state=100,n_jobs=-1, verbose=2)

estimators.append(('rf2', rfclassifer2))

rfclassifer3 = RandomForestClassifier(n_estimators = 150,
               bootstrap = True,random_state=4833,n_jobs=-1, verbose=2)

estimators.append(('rf3', rfclassifer3))

ensemblerfVoting = VotingClassifier(estimators, voting='soft', n_jobs=-1)

In [13]:
ensemblerfVoting.fit(xTrain, yTrain)

error: 'i' format requires -2147483648 <= number <= 2147483647

In [14]:
ensemblerfPred = ensemblerfVoting.predict_proba(xTest,)

/home/mpghare/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:356: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/mpghare/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [15]:
log_loss(yTest, ensemblerfPred, eps=1e-15, normalize=True)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
from sklearn.calibration import CalibratedClassifierCV

calibrated_rf_clf = CalibratedClassifierCV(ensemblerfVoting, method='isotonic', cv=5)
calibrated_rf_clf.fit(xTrain, yTrain)
rfCalibEnsemblePred = calibrated_rf_clf.predict_proba(xTest)

print("loss WITH calibration : ", log_loss(yTest, rfCalibEnsemblePred, eps=1e-15, normalize=True))

In [ ]:
ensemblerfPredVals = ensemblerfVoting.predict(xTest,)

In [ ]:
print("AUC Score : %f" % metrics.roc_auc_score(yTest, ensemblerfPred[:,1]))
print("Accuracy : %.4g" % metrics.accuracy_score(yTest, ensemblerfPredVals))

In [ ]:
# save the model to disk
filename = 'Ensemble_RF_model.sav'
pickle.dump(ensemblerfVoting, open(filename, 'wb'))

## XG Boost

In [37]:
dTrain = xgb.DMatrix(xTrain, yTrain)

In [54]:
print(dTrain)

In [40]:
st.randint(120, 1200)

In [39]:
one_to_left = st.beta(10, 1)  
from_zero_positive = st.expon(0, 50)

alg = XGBClassifier(
 learning_rate = st.uniform(0.05, 0.4),
 n_estimators=st.randint(120, 1200),
 max_depth=st.randint(3, 40),
 min_child_weight=from_zero_positive,
 reg_alpha = from_zero_positive,
 gamma=st.uniform(0, 10),
 subsample = one_to_left,
 colsample_bytree = one_to_left,
 nthreads=-1,
    n_jobs=-1,
 objective='multi:softprob',
 stratified=True,
 scale_pos_weight=1, 
 num_class = 2,   
 seed=5381)

# xgb1_model = modelfit(xgb1, xTrain, predictors)
xgb_param = alg.get_xgb_params()
cvresult = xgb.cv(xgb_param, dTrain, num_boost_round=alg.get_params()['n_estimators'], \
                  nfold=5, metrics='mlogloss', early_stopping_rounds=50, verbose_eval=False)
alg.set_params(n_estimators=cvresult.shape[0])
    
#Fit the algorithm on the data
mdl = alg.fit(xTrain, label_encoded_y, eval_metric='logloss')

TypeError: 'rv_frozen' object cannot be interpreted as an integer

In [42]:
params_xgb = {
    "learning_rate" : st.uniform(0.05, 0.4),
 "n_estimators":st.randint(120, 1200),
 "max_depth":st.randint(3, 40),
 "min_child_weight":from_zero_positive,
 "reg_alpha" : from_zero_positive,
 "gamma":st.uniform(0, 10),
 "subsample" : one_to_left,
 "colsample_bytree" : one_to_left,
 "nthreads":-1,
    "njobs":-1,
 "objective":'multi:softprob',
 "stratified":True,
 "scale_pos_weight":1, 
 "num_class" : 2,   
 "seed":5381
}

In [58]:
params_xgb

{'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen at 0x7fe4ca9d4b70>,
 'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen at 0x7fe4c68846d8>,
 'max_depth': <scipy.stats._distn_infrastructure.rv_frozen at 0x7fe4c8707e80>,
 'min_child_weight': <scipy.stats._distn_infrastructure.rv_frozen at 0x7fe4ca75b2b0>,
 'reg_alpha': <scipy.stats._distn_infrastructure.rv_frozen at 0x7fe4ca75b2b0>,
 'gamma': <scipy.stats._distn_infrastructure.rv_frozen at 0x7fe4c6394ac8>,
 'subsample': <scipy.stats._distn_infrastructure.rv_frozen at 0x7fe4ca75b128>,
 'colsample_bytree': <scipy.stats._distn_infrastructure.rv_frozen at 0x7fe4ca75b128>,
 'nthreads': -1,
 'njobs': -1,
 'objective': 'multi:softprob',
 'stratified': True,
 'scale_pos_weight': 1,
 'num_class': 2,
 'seed': 5381}

In [44]:
xbgClassifier = XGBClassifier(nthreads=-1)

In [45]:
gs = RandomizedSearchCV(xbgClassifier, params_xgb, n_jobs=1)  

In [55]:
grid_xgb = gs.fit(xTrain, yTrain, eval_metric='logloss')

/home/mpghare/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


TypeError: object of type 'int' has no len()

In [ ]:
mdl.get_params

In [ ]:
xgb_pred = mdl.predict_proba(xTest,)

In [ ]:
log_loss(yTest, xgb1_pred)

In [ ]:
# save the model to disk
filename = 'XGB_model.sav'
pickle.dump(mdl, open(filename, 'wb'))

In [145]:
xTrainNew = xTrain.loc[:, ["ProductName", "EngineVersion", "AppVersion", "AvSigVersion", "AVProductStatesIdentifier", "HasTpm", "CountryIdentifier", "CityIdentifier", "OrganizationIdentifier", "GeoNameIdentifier", "Platform", "Processor", "IsProtected", "PuaMode", "SMode", "Firewall", "UacLuaenable", "Census_MDC2FormFactor", "Census_DeviceFamily", "Census_ProcessorCoreCount", "Census_ProcessorClass", "Census_PrimaryDiskTotalCapacity", "Census_PrimaryDiskTypeName", "Census_SystemVolumeTotalCapacity", "Census_HasOpticalDiskDrive", "Census_TotalPhysicalRAM", "Census_PowerPlatformRoleName", "Census_OSVersion", "Census_OSArchitecture", "Census_OSBranch", "Census_OSBuildNumber", "Census_OSBuildRevision", "Census_OSEdition", "Census_OSSkuName", "Census_OSInstallTypeName", "Census_OSWUAutoUpdateOptionsName", "Census_IsPortableOperatingSystem", "Census_GenuineStateName", "Census_IsFlightsDisabled", "Census_FlightRing", "Census_IsSecureBootEnabled", "Census_IsVirtualDevice", "Census_IsAlwaysOnAlwaysConnectedCapable", "Wdft_IsGamer"]]

In [146]:
xTrainNew.head()

,ProductName,EngineVersion,AppVersion,AvSigVersion,AVProductStatesIdentifier,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,...,Census_OSInstallTypeName,Census_OSWUAutoUpdateOptionsName,Census_IsPortableOperatingSystem,Census_GenuineStateName,Census_IsFlightsDisabled,Census_FlightRing,Census_IsSecureBootEnabled,Census_IsVirtualDevice,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer
7587898,4,65,57,8135,53447.0,1,168,-999.0,-999.0,89.0,...,8,2,0,1,0.0,6,0,0.0,0.0,1.0
3904626,1,65,106,8328,27750.0,0,199,150207.0,27.0,89.0,...,7,2,0,1,0.0,6,0,0.0,0.0,0.0
8705769,4,65,57,8088,53447.0,1,158,-999.0,-999.0,202.0,...,7,3,0,1,0.0,6,1,0.0,0.0,1.0
1343039,4,64,57,7680,7945.0,1,9,20805.0,27.0,240.0,...,7,3,0,1,0.0,6,0,0.0,0.0,1.0
7183243,4,65,57,8038,53447.0,1,60,10653.0,27.0,240.0,...,7,0,0,1,0.0,6,0,0.0,0.0,0.0


In [147]:
xTestNew = xTest.loc[:, ["ProductName", "EngineVersion", "AppVersion", "AvSigVersion", "AVProductStatesIdentifier", "HasTpm", "CountryIdentifier", "CityIdentifier", "OrganizationIdentifier", "GeoNameIdentifier", "Platform", "Processor", "IsProtected", "PuaMode", "SMode", "Firewall", "UacLuaenable", "Census_MDC2FormFactor", "Census_DeviceFamily", "Census_ProcessorCoreCount", "Census_ProcessorClass", "Census_PrimaryDiskTotalCapacity", "Census_PrimaryDiskTypeName", "Census_SystemVolumeTotalCapacity", "Census_HasOpticalDiskDrive", "Census_TotalPhysicalRAM", "Census_PowerPlatformRoleName", "Census_OSVersion", "Census_OSArchitecture", "Census_OSBranch", "Census_OSBuildNumber", "Census_OSBuildRevision", "Census_OSEdition", "Census_OSSkuName", "Census_OSInstallTypeName", "Census_OSWUAutoUpdateOptionsName", "Census_IsPortableOperatingSystem", "Census_GenuineStateName", "Census_IsFlightsDisabled", "Census_FlightRing", "Census_IsSecureBootEnabled", "Census_IsVirtualDevice", "Census_IsAlwaysOnAlwaysConnectedCapable", "Wdft_IsGamer"]]

In [141]:
clf = xgb.XGBClassifier(max_depth=11,
                           min_child_weight=1,
                           learning_rate=0.1,
                           n_estimators=100,
                           silent=True,
                           objective='binary:logistic',
                           gamma=0,
                           max_delta_step=0,
                           subsample=0.5,
                           colsample_bytree=1,
                           colsample_bylevel=1,
                           reg_alpha=0,
                           reg_lambda=0,
                           scale_pos_weight=1,
                           seed=1,
                           missing=None,
                           n_jobs=-1,
                           nthreads=-1)

clf.fit(xTrain, yTrain, eval_metric='auc', eval_set=[(xTest, yTest)], verbose=True, early_stopping_rounds=100)

[18:58:34] Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
[0]	validation_0-auc:0.612075
Will train until validation_0-auc hasn't improved in 100 rounds.
[1]	validation_0-auc:0.628065
[2]	validation_0-auc:0.63243
[3]	validation_0-auc:0.632912
[4]	validation_0-auc:0.631555
[5]	validation_0-auc:0.637196
[6]	validation_0-auc:0.643063
[7]	validation_0-auc:0.645267
[8]	validation_0-auc:0.644758
[9]	validation_0-auc:0.642319
[10]	validation_0-auc:0.643777
[11]	validation_0-auc:0.647125
[12]	validation_0-auc:0.650157
[13]	validation_0-auc:0.652766
[14]	validation_0-auc:0.652978
[15]	validation_0-auc:0.651692
[16]	validation_0-auc:0.654254
[17]	validation_0-auc:0.654485
[18]	validation_0-auc:0.656278
[19]	validation_0-auc:0.65679
[20]	validation_0-auc:0.658552
[21]	validation_0-auc:0.659441
[22]	validation_0-auc:0.660024
[23]	validation_0-auc:0.660794
[24]	validation_0-auc:0.66107

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=11, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=-1, nthread=None, nthreads=-1, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=0, scale_pos_weight=1,
       seed=1, silent=True, subsample=0.5)

In [120]:
# load the model from disk
clf = pickle.load(open('XGB_model.sav', 'rb'))

In [142]:
y_pre = clf.predict(xTest)
y_pro = clf.predict_proba(xTest)[:, 1]

In [143]:
print("AUC Score : %f" % metrics.roc_auc_score(yTest, y_pro))
print("Accuracy : %.4g" % metrics.accuracy_score(yTest, y_pre))

AUC Score : 0.661074
Accuracy : 0.5932


In [33]:

# save the model to disk
filename = 'XGB_model.sav'
pickle.dump(clf, open(filename, 'wb'))

## Light GBM

In [11]:
train_data = lgb.Dataset(xTrain, label=yTrain)
test_data = lgb.Dataset(xTest, label=yTest)

In [148]:
train_data_new = lgb.Dataset(xTrainNew, label=yTrain)
test_data_new = lgb.Dataset(xTestNew, label=yTest)

In [29]:
param = {'num_leaves':43, 
         'num_round': 500,
         'num_trees':5000, 
         'objective':'binary',
         'learning_rate': 0.025,
         'max_depth': 31,
         'seed': 1324,
         'num_threads': 64, 
         'verbose': 2}
param['metric'] = 'auc'

In [30]:
#lgb.cv(param, train_data, num_round, nfold=5)
bst = lgb.train(param, train_data, valid_sets=[test_data])

[1]	valid_0's auc: 0.625226
[2]	valid_0's auc: 0.625226
[3]	valid_0's auc: 0.625225
[4]	valid_0's auc: 0.625225
[5]	valid_0's auc: 0.626051
[6]	valid_0's auc: 0.626519
[7]	valid_0's auc: 0.632676
[8]	valid_0's auc: 0.632645
[9]	valid_0's auc: 0.632966
[10]	valid_0's auc: 0.633469
[11]	valid_0's auc: 0.633462
[12]	valid_0's auc: 0.633914
[13]	valid_0's auc: 0.635114
[14]	valid_0's auc: 0.635429
[15]	valid_0's auc: 0.635574
[16]	valid_0's auc: 0.635906
[17]	valid_0's auc: 0.635784
[18]	valid_0's auc: 0.635815
[19]	valid_0's auc: 0.637209
[20]	valid_0's auc: 0.637678
[21]	valid_0's auc: 0.637928
[22]	valid_0's auc: 0.637813
[23]	valid_0's auc: 0.638366
[24]	valid_0's auc: 0.639119
[25]	valid_0's auc: 0.639159
[26]	valid_0's auc: 0.639692
[27]	valid_0's auc: 0.639839
[28]	valid_0's auc: 0.640316
[29]	valid_0's auc: 0.640676
[30]	valid_0's auc: 0.640395
[31]	valid_0's auc: 0.641456
[32]	valid_0's auc: 0.641552
[33]	valid_0's auc: 0.641624
[34]	valid_0's auc: 0.642191
[35]	valid_0's auc: 0.6

KeyboardInterrupt: 

In [52]:
bst.save_model('lightGBMmodel.txt')

In [29]:
bst = lgb.Booster(model_file='lightGBMmodel.txt')  #init model

In [36]:
result_gbm = bst.predict(xTest)

In [37]:
print(result_gbm)

[0.29018533 0.27479579 0.24027571 ... 0.44725689 0.29124064 0.29670322]


In [38]:
print("AUC Score : %f" % metrics.roc_auc_score(yTest, result_gbm))

AUC Score : 0.681284


In [16]:
param_ens = {'num_leaves':43, 
         'num_round': 500,
         'num_trees':500, 
         'objective':'binary',
         'learning_rate': 0.03,
         'max_depth': 31,
         'seed': 1324,
         'num_threads': 64, 
         'verbose': 2}
param_ens['metric'] = 'auc'

In [17]:
bst1 = lgb.train(param_ens, train_data, valid_sets=[test_data])

[1]	valid_0's auc: 0.625226
[2]	valid_0's auc: 0.625226
[3]	valid_0's auc: 0.625225
[4]	valid_0's auc: 0.625942
[5]	valid_0's auc: 0.625925
[6]	valid_0's auc: 0.632115
[7]	valid_0's auc: 0.63292
[8]	valid_0's auc: 0.633381
[9]	valid_0's auc: 0.633095
[10]	valid_0's auc: 0.633666
[11]	valid_0's auc: 0.635356
[12]	valid_0's auc: 0.633999
[13]	valid_0's auc: 0.634326
[14]	valid_0's auc: 0.634765
[15]	valid_0's auc: 0.635697
[16]	valid_0's auc: 0.636368
[17]	valid_0's auc: 0.636938
[18]	valid_0's auc: 0.637946
[19]	valid_0's auc: 0.638523
[20]	valid_0's auc: 0.639134
[21]	valid_0's auc: 0.639117
[22]	valid_0's auc: 0.640592
[23]	valid_0's auc: 0.640658
[24]	valid_0's auc: 0.64096
[25]	valid_0's auc: 0.641496
[26]	valid_0's auc: 0.641439
[27]	valid_0's auc: 0.64164
[28]	valid_0's auc: 0.640855
[29]	valid_0's auc: 0.642053
[30]	valid_0's auc: 0.643446
[31]	valid_0's auc: 0.644125
[32]	valid_0's auc: 0.6461
[33]	valid_0's auc: 0.647101
[34]	valid_0's auc: 0.647116
[35]	valid_0's auc: 0.647502

In [32]:
param_ens2 = {'num_leaves':35, 
         'num_round': 500,
         'num_trees':500, 
         'objective':'binary',
         'learning_rate': 0.07,
         'max_depth': 35,
         'seed': 719,
         'num_threads': 64, 
         'verbose': 2}
param_ens2['metric'] = 'auc'

In [35]:
bst2 = lgb.train(param_ens2, train_data, valid_sets=[test_data])

[1]	valid_0's auc: 0.620001
[2]	valid_0's auc: 0.619999
[3]	valid_0's auc: 0.629799
[4]	valid_0's auc: 0.629187
[5]	valid_0's auc: 0.632037
[6]	valid_0's auc: 0.635906
[7]	valid_0's auc: 0.63619
[8]	valid_0's auc: 0.637774
[9]	valid_0's auc: 0.638827
[10]	valid_0's auc: 0.639632
[11]	valid_0's auc: 0.639366
[12]	valid_0's auc: 0.638551
[13]	valid_0's auc: 0.641056
[14]	valid_0's auc: 0.641022
[15]	valid_0's auc: 0.643413
[16]	valid_0's auc: 0.645978
[17]	valid_0's auc: 0.647924
[18]	valid_0's auc: 0.649597
[19]	valid_0's auc: 0.649651
[20]	valid_0's auc: 0.651657
[21]	valid_0's auc: 0.653372
[22]	valid_0's auc: 0.654389
[23]	valid_0's auc: 0.654268
[24]	valid_0's auc: 0.65419
[25]	valid_0's auc: 0.655213
[26]	valid_0's auc: 0.655122
[27]	valid_0's auc: 0.656067
[28]	valid_0's auc: 0.655853
[29]	valid_0's auc: 0.657001
[30]	valid_0's auc: 0.658342
[31]	valid_0's auc: 0.658514
[32]	valid_0's auc: 0.6584
[33]	valid_0's auc: 0.658966
[34]	valid_0's auc: 0.658852
[35]	valid_0's auc: 0.65951

In [41]:
param_ens3 = {'num_leaves':39, 
         'num_round': 500,
         'num_trees':700, 
         'objective':'binary',
         'learning_rate': 0.07,
         'max_depth': 37,
         'seed': 719,
         'num_threads': 64, 
         'verbose': 2}
param_ens3['metric'] = 'auc'

In [42]:
bst3 = lgb.train(param_ens3, train_data, valid_sets=[test_data])

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:116: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's auc: 0.620001
[2]	valid_0's auc: 0.619999
[3]	valid_0's auc: 0.631092
[4]	valid_0's auc: 0.63102
[5]	valid_0's auc: 0.63318
[6]	valid_0's auc: 0.636211
[7]	valid_0's auc: 0.636716
[8]	valid_0's auc: 0.637295
[9]	valid_0's auc: 0.639306
[10]	valid_0's auc: 0.640865
[11]	valid_0's auc: 0.640583
[12]	valid_0's auc: 0.641805
[13]	valid_0's auc: 0.644944
[14]	valid_0's auc: 0.64618
[15]	valid_0's auc: 0.647016
[16]	valid_0's auc: 0.648583
[17]	valid_0's auc: 0.650201
[18]	valid_0's auc: 0.649706
[19]	valid_0's auc: 0.651072
[20]	valid_0's auc: 0.652928
[21]	valid_0's auc: 0.654135
[22]	valid_0's auc: 0.653853
[23]	valid_0's auc: 0.655296
[24]	valid_0's auc: 0.65613
[25]	valid_0's auc: 0.656114
[26]	valid_0's auc: 0.657213
[27]	valid_0's auc: 0.657654
[28]	valid_0's auc: 0.658472
[29]	valid_0's auc: 0.658657
[30]	valid_0's auc: 0.659629
[31]	valid_0's auc: 0.659465
[32]	valid_0's auc: 0.660057
[33]	valid_0's auc: 0.659695
[34]	valid_0's auc: 0.660409
[35]	valid_0's auc: 0.66096

In [45]:
param_ens4 = {'num_leaves':47, 
         'num_round': 500,
         'num_trees':550, 
         'objective':'binary',
         'learning_rate': 0.07,
         'max_depth': 31,
         'seed': 1085,
         'num_threads': 64, 
         'verbose': 2}
param_ens4['metric'] = 'auc'

In [46]:
bst4 = lgb.train(param_ens4, train_data, valid_sets=[test_data])

[1]	valid_0's auc: 0.625226
[2]	valid_0's auc: 0.625666
[3]	valid_0's auc: 0.632751
[4]	valid_0's auc: 0.632499
[5]	valid_0's auc: 0.634337
[6]	valid_0's auc: 0.636884
[7]	valid_0's auc: 0.636262
[8]	valid_0's auc: 0.637486
[9]	valid_0's auc: 0.639742
[10]	valid_0's auc: 0.639981
[11]	valid_0's auc: 0.640924
[12]	valid_0's auc: 0.642496
[13]	valid_0's auc: 0.642535
[14]	valid_0's auc: 0.644244
[15]	valid_0's auc: 0.647958
[16]	valid_0's auc: 0.648447
[17]	valid_0's auc: 0.648702
[18]	valid_0's auc: 0.648837
[19]	valid_0's auc: 0.651357
[20]	valid_0's auc: 0.652336
[21]	valid_0's auc: 0.654333
[22]	valid_0's auc: 0.654719
[23]	valid_0's auc: 0.656582
[24]	valid_0's auc: 0.657323
[25]	valid_0's auc: 0.65838
[26]	valid_0's auc: 0.658663
[27]	valid_0's auc: 0.659814
[28]	valid_0's auc: 0.659536
[29]	valid_0's auc: 0.660337
[30]	valid_0's auc: 0.661021
[31]	valid_0's auc: 0.660702
[32]	valid_0's auc: 0.661236
[33]	valid_0's auc: 0.661415
[34]	valid_0's auc: 0.661861
[35]	valid_0's auc: 0.66

In [47]:
result_gbm1 = bst1.predict(xTest)
result_gbm2 = bst2.predict(xTest)
result_gbm3 = bst3.predict(xTest)
result_gbm4 = bst4.predict(xTest)

In [48]:
result_gbm_ens = (result_gbm1 + result_gbm2 + result_gbm3 + result_gbm4)/4

In [49]:
print("AUC Score : %f" % metrics.roc_auc_score(yTest, result_gbm_ens))

AUC Score : 0.684849


In [55]:
fin_result_gbm1 = bst1.predict(DataTestFin)
fin_result_gbm2 = bst2.predict(DataTestFin)
fin_result_gbm3 = bst3.predict(DataTestFin)
fin_result_gbm4 = bst4.predict(DataTestFin)

In [56]:
fin_gbmpro = (fin_result_gbm1 + fin_result_gbm2 + fin_result_gbm3 + fin_result_gbm4)/4

In [57]:
results_final = pd.DataFrame(data=fin_gbmpro,    # values
           index=DataTest['MachineIdentifier'],    # 1st column as index
           columns=['HasDetections'])  # 1st row as the column names

In [58]:
pd.DataFrame.to_csv(results_final, "results_final_lightgbm_ens.csv")

## Random Forest

In [12]:
rfClf = RandomForestClassifier(n_estimators = 150, random_state=6, n_jobs=-1, verbose=2)

In [13]:
rfClf.fit(xTrain, yTrain)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 64 concurrent workers.


building tree 1 of 150
building tree 2 of 150
building tree 3 of 150building tree 4 of 150
building tree 5 of 150
building tree 6 of 150
building tree 7 of 150
building tree 8 of 150
building tree 9 of 150
building tree 10 of 150
building tree 11 of 150
building tree 12 of 150
building tree 13 of 150
building tree 14 of 150
building tree 15 of 150
building tree 16 of 150
building tree 17 of 150
building tree 18 of 150
building tree 19 of 150
building tree 20 of 150
building tree 21 of 150
building tree 22 of 150
building tree 23 of 150
building tree 24 of 150
building tree 25 of 150
building tree 26 of 150
building tree 27 of 150
building tree 28 of 150
building tree 29 of 150
building tree 30 of 150
building tree 31 of 150
building tree 32 of 150
building tree 33 of 150
building tree 34 of 150
building tree 35 of 150building tree 36 of 150
building tree 37 of 150
building tree 38 of 150
building tree 39 of 150
building tree 40 of 150

building tree 41 of 150
building tree 42 of 150
bu

[Parallel(n_jobs=-1)]: Done  99 out of 150 | elapsed:  3.9min remaining:  2.0min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  5.2min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=-1,
            oob_score=False, random_state=6, verbose=2, warm_start=False)

In [14]:
y_rf_pre = rfClf.predict(xTest)
y_rf_pro = rfClf.predict_proba(xTest)[:, 1]

[Parallel(n_jobs=64)]: Using backend ThreadingBackend with 64 concurrent workers.
[Parallel(n_jobs=64)]: Done  99 out of 150 | elapsed:    7.7s remaining:    4.0s
[Parallel(n_jobs=64)]: Done 150 out of 150 | elapsed:   10.2s finished
[Parallel(n_jobs=64)]: Using backend ThreadingBackend with 64 concurrent workers.
[Parallel(n_jobs=64)]: Done  99 out of 150 | elapsed:    7.6s remaining:    3.9s
[Parallel(n_jobs=64)]: Done 150 out of 150 | elapsed:   10.3s finished


In [15]:
print("AUC Score : %f" % metrics.roc_auc_score(yTest, y_rf_pro))
print("Accuracy : %.4g" % metrics.accuracy_score(yTest, y_rf_pre))

AUC Score : 0.660600
Accuracy : 0.5925


In [88]:
# save the model to disk
filename = 'RF_model.sav'
pickle.dump(rfClf, open(filename, 'wb'))

In [77]:
rfClf2 = pickle.load(open('RF_model.sav', 'rb'))

KeyboardInterrupt: 

In [80]:
rfClf2 = RandomForestClassifier(n_estimators = 200, random_state=8923, n_jobs=-1, verbose=2)

In [81]:
rfClf2.fit(xTrain, yTrain)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 64 concurrent workers.


building tree 1 of 200
building tree 2 of 200
building tree 3 of 200
building tree 4 of 200
building tree 5 of 200
building tree 6 of 200
building tree 7 of 200
building tree 8 of 200
building tree 9 of 200
building tree 10 of 200
building tree 11 of 200building tree 12 of 200
building tree 13 of 200
building tree 14 of 200
building tree 15 of 200
building tree 16 of 200
building tree 17 of 200
building tree 18 of 200
building tree 19 of 200
building tree 20 of 200
building tree 21 of 200
building tree 22 of 200
building tree 23 of 200
building tree 24 of 200
building tree 25 of 200
building tree 26 of 200
building tree 27 of 200
building tree 28 of 200
building tree 29 of 200
building tree 30 of 200
building tree 31 of 200
building tree 32 of 200
building tree 33 of 200
building tree 34 of 200
building tree 35 of 200building tree 36 of 200
building tree 37 of 200
building tree 38 of 200
building tree 39 of 200

building tree 40 of 200building tree 41 of 200
building tree 42 of 200
bui

[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  3.0min


building tree 101 of 200
building tree 102 of 200building tree 103 of 200

building tree 104 of 200
building tree 105 of 200
building tree 106 of 200
building tree 107 of 200
building tree 108 of 200
building tree 109 of 200
building tree 110 of 200
building tree 111 of 200
building tree 112 of 200
building tree 113 of 200
building tree 114 of 200building tree 115 of 200
building tree 116 of 200

building tree 117 of 200
building tree 118 of 200
building tree 119 of 200
building tree 120 of 200
building tree 121 of 200
building tree 122 of 200building tree 123 of 200

building tree 124 of 200
building tree 125 of 200
building tree 126 of 200
building tree 127 of 200
building tree 128 of 200
building tree 129 of 200
building tree 130 of 200
building tree 131 of 200
building tree 132 of 200
building tree 133 of 200
building tree 134 of 200
building tree 135 of 200
building tree 136 of 200
building tree 137 of 200
building tree 138 of 200
building tree 139 of 200
building tree 140 of 200


[Parallel(n_jobs=-1)]: Done 174 out of 200 | elapsed:  6.6min remaining:   59.6s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:  7.6min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
            oob_score=False, random_state=8923, verbose=2,
            warm_start=False)

In [82]:
y_rf_pre2 = rfClf2.predict(xTest)
y_rf_pro2 = rfClf2.predict_proba(xTest)[:, 1]

[Parallel(n_jobs=64)]: Using backend ThreadingBackend with 64 concurrent workers.
[Parallel(n_jobs=64)]: Done  34 tasks      | elapsed:    3.9s
[Parallel(n_jobs=64)]: Done 174 out of 200 | elapsed:   11.2s remaining:    1.7s
[Parallel(n_jobs=64)]: Done 200 out of 200 | elapsed:   12.8s finished
[Parallel(n_jobs=64)]: Using backend ThreadingBackend with 64 concurrent workers.
[Parallel(n_jobs=64)]: Done  34 tasks      | elapsed:    3.8s
[Parallel(n_jobs=64)]: Done 174 out of 200 | elapsed:   11.3s remaining:    1.7s
[Parallel(n_jobs=64)]: Done 200 out of 200 | elapsed:   12.5s finished


In [83]:
print("AUC Score : %f" % metrics.roc_auc_score(yTest, y_rf_pro2))
print("Accuracy : %.4g" % metrics.accuracy_score(yTest, y_rf_pre2))

AUC Score : 0.661817
Accuracy : 0.5921


In [157]:
DataTestNew = DataTest.loc[:, ["ProductName", "EngineVersion", "AppVersion", "AvSigVersion", "AVProductStatesIdentifier", "HasTpm", "CountryIdentifier", "CityIdentifier", "OrganizationIdentifier", "GeoNameIdentifier", "LocaleEnglishNameIdentifier", "Platform", "Processor", "OsVer", "OsBuild", "OsSuite", "OsPlatformSubRelease", "OsBuildLab", "SkuEdition", "IsProtected", "PuaMode", "SMode", "SmartScreen", "Firewall", "UacLuaenable", "Census_MDC2FormFactor", "Census_DeviceFamily", "Census_ProcessorCoreCount", "Census_ProcessorClass", "Census_PrimaryDiskTotalCapacity", "Census_PrimaryDiskTypeName", "Census_SystemVolumeTotalCapacity", "Census_HasOpticalDiskDrive", "Census_TotalPhysicalRAM", "Census_ChassisTypeName", "Census_InternalPrimaryDiagonalDisplaySizeInInches", "Census_InternalPrimaryDisplayResolutionHorizontal", "Census_InternalPrimaryDisplayResolutionVertical", "Census_PowerPlatformRoleName", "Census_OSVersion", "Census_OSArchitecture", "Census_OSBranch", "Census_OSBuildNumber", "Census_OSBuildRevision", "Census_OSEdition", "Census_OSSkuName", "Census_OSInstallTypeName", "Census_OSWUAutoUpdateOptionsName", "Census_IsPortableOperatingSystem", "Census_GenuineStateName", "Census_ActivationChannel", "Census_IsFlightsDisabled", "Census_FlightRing", "Census_IsSecureBootEnabled", "Census_IsVirtualDevice", "Census_IsTouchEnabled", "Census_IsPenCapable", "Census_IsAlwaysOnAlwaysConnectedCapable", "Wdft_IsGamer"]]

In [159]:
le = LabelEncoder()
for column_name in DataTestNew.columns:
        if DataTestNew[column_name].dtype == object:
            DataTestNew.loc[:, column_name] = le.fit_transform(DataTestNew[column_name].astype(str))
        else:
            pass

In [160]:
DataTestNew = DataTestNew.fillna(-999)

In [124]:
en_pro = (result_gbm + y_pro)/2

In [125]:
print(en_pro)

[0.32671932 0.33783777 0.30276918 ... 0.48044663 0.32150615 0.3035524 ]


In [129]:
print("AUC Score : %f" % metrics.roc_auc_score(yTest, en_pro))

AUC Score : 0.680507


In [123]:
testy_pro = clf.predict_proba(DataTest)[:, 1]
#testy_rfpro = rfClf.predict_proba(DataTest)[:, 1]

In [130]:
print(testy_pro)

[0.49543533 0.523181   0.3986612  ... 0.35402623 0.50318897 0.2715728 ]


In [57]:
testy_gbmpro = bst.predict(DataTest)

In [58]:
print(testy_gbmpro)

[0.55866487 0.6029267  0.41654932 ... 0.27453316 0.50129568 0.32905048]


In [133]:
test_en_pro = (testy_pro + testy_gbmpro)/2

In [134]:
print(test_en_pro)

[0.49817061 0.5499676  0.39123348 ... 0.32714223 0.49561143 0.30365305]


In [63]:
results_final = pd.DataFrame(data=testy_gbmpro,    # values
           index=DataTest['MachineIdentifier'],    # 1st column as index
           columns=['HasDetections'])  # 1st row as the column names

In [64]:
print(results_final)

                                  HasDetections
MachineIdentifier                              
0000010489e3af074adeac69c53e555e       0.558665
00000176ac758d54827acd545b6315a5       0.602927
0000019dcefc128c2d4387c1273dae1d       0.416549
0000055553dc51b1295785415f1a224d       0.388210
00000574cefffeca83ec8adf9285b2bf       0.464104
000007ffedd31948f08e6c16da31f6d1       0.440256
000008f31610018d898e5f315cdf1bd1       0.131185
00000a3c447250626dbcc628c9cbc460       0.229086
00000b6bf217ec9aef0f68d5c6705897       0.411359
00000b8d3776b13e93ad83676a28e4aa       0.376459
00000dec341e29f26b92c3be03640bdc       0.197773
00000e658ce75c1e2a3bb47bcc3b08f3       0.510399
0000102ff65968bbdc04b69073434b05       0.440849
000011236a5dc4ff119541c42bb4287e       0.715456
0000124d8811c1a5b5848c4d730cfbf8       0.394258
000015a88b949c982bc80ec8e32b726d       0.489742
0000187a73c37cab54bd5be81c16c71f       0.323330
00001be80a499e95d5e68151b6470aa2       0.294851
00001dcfc3f82d68d6eae9cad4a3e07c       0

In [65]:
pd.DataFrame.to_csv(results_final, "results_final_lightgbm2.csv")